In [1]:
# Import necessary packages
import ee
import geemap


In [2]:
# Initiate interactive map
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('ROADMAP') # Add Google Map
Map

layer already on map: TileLayer(attribution='Google', name='Google Maps', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
Basemap can only be one of the following:
  ROADMAP
  SATELLITE
  TERRAIN
  HYBRID
  ESRI
  Esri Ocean
  Esri Satellite
  Esri Standard
  Esri Terrain
  Esri Transportation
  Esri Topo World
  Esri National Geographic
  Esri Shaded Relief
  Esri Physical Map
  FWS NWI Wetlands
  FWS NWI Wetlands Raster
  Google Maps
  Google Satellite
  Google Terrain
  Google Satellite Hybrid
  NLCD 2016 CONUS Land Cover
  NLCD 2013 CONUS Land Cover
  NLCD 2011 CONUS Land Cover
  NLCD 2008 CONUS Land Cover
  NLCD 2006 CONUS Land Cover
  NLCD 2004 CONUS Land Cover
  NLCD 2001 CONUS Land Cover
  USGS NAIP Imagery
  USGS Hydrography
  USGS 3DEP Elevation
  OpenStreetMap.Mapnik
  OpenStreetMap.BlackAndWhite
  OpenStreetMap.DE
  OpenStreetMap.F

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# If you want to define a different ROI with inspector tool
# roi = ee.FeatureCollection(Map.draw_features)

# Define ROI polygon
roi = ee.Geometry.Polygon([[[-105.03081, 39.311584],
      [-104.569348, 39.320083],
      [-104.497931, 39.517411],
      [-104.500678, 39.98619],
      [-104.835787, 40.059806],
      [-105.187377, 39.939877],
      [-105.228579, 39.743753],
      [-105.03081, 39.311584]]])

In [ ]:
# Visualizing L7 form data catalog
# ee.ImageCollection(
#     'LANDSAT/LE07/C01/T1_TOA').filterDate('1999-01-01', '2002-12-31')
# trueColor321 = dataset.select(['B3', 'B2', 'B1'])
# trueColor321Vis = {
#     min: 0.0,
#     max: 0.4,
#     gamma: 1.2}


##### Before 2013 we will be using Landsat 5 - not landsat 7.   Therefore these next two cells need to be converted to Landsat 5

In [ ]:
# Can I visualize a single L7 scene?
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
d1, d2 = '1999-01-01', '1999-12-31'
cloudFree = l7.filterDate(d1, d2).filterBounds(roi).median()

clipped_cloudFree = cloudFree.clip(roi)

# Add L7 RGB to map
visParams = {'bands': ['B3', 'B2', 'B1'],
  'min': 0.0,
  'max': 0.4,
  'gamma': 1.2}

Map.addLayer(clipped_cloudFree, visParams, 'L7_clipped')

In [ ]:
# Function to clip cloud-free Landsat 7 to given area for given year(s)
def cloudFreeClipped_l7(year, area):
    # Get raw Landsat7 data input for simple composite
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')

    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-01-01', str(year)+'-12-31'

    # Filter Landsat7 collection by date and by ROI then reduce to ee.Image by median
    cloudFree = l7.filterDate(d1, d2).filterBounds(area).median()

    # Clip to ROI
    clipped_cloudFree = cloudFree.clip(area)

    # Add L7 RGB to map
    visParams = {'bands': ['B3', 'B2', 'B1'],
      'min': 0.0,
      'max': 0.4,
      'gamma': 1.2}

    Map.addLayer(clipped_cloudFree, visParams, str(year))

## Landsat 8

In [4]:
# Function to get NDVI for Landsat 8 ee.Image object
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [5]:
# Function to clip cloud-free Landsat 8 to given area for given year(s)
def cloudFreeClipped_l8(year, area):
    # Get raw Landsat8 data input for simple composite
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
    
    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-07-01', str(year)+'-09-30'

    # Filter Landsat8 collection by date and by ROI
    collection = l8.filterDate(d1, d2).filterBounds(area)
    print('collectionl8', type(collection))
    
#     # Map over NDVI function
#     withNDVI = collection.map(addNDVI)
    
    
    # Create cloud-free composite with default params
    composite = ee.Algorithms.Landsat.simpleComposite(collection)
    print('compositel8', type(composite))
    
    ndvi = addNDVI(composite).select('NDVI')
    
    # Clip to ROI
    clippedComposite = ndvi.clip(area)
    print('clippedCompositel8', type(clippedComposite))
    
    ndviParams = {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    
    # Display the clipped composite as RGB
    Map.addLayer(clippedComposite, ndviParams, str(year))
    
    return clippedComposite

In [6]:
# Add Landsat8 to map 
for i in range(2013, 2016):
    cloudFreeClipped_l8(i, roi)
    


collectionl8 <class 'ee.imagecollection.ImageCollection'>
compositel8 <class 'ee.image.Image'>
clippedCompositel8 <class 'ee.image.Image'>
collectionl8 <class 'ee.imagecollection.ImageCollection'>
compositel8 <class 'ee.image.Image'>
clippedCompositel8 <class 'ee.image.Image'>
collectionl8 <class 'ee.imagecollection.ImageCollection'>
compositel8 <class 'ee.image.Image'>
clippedCompositel8 <class 'ee.image.Image'>


In [7]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
# Add Landsat5 to Map
for i in range (1999, 2014):
    cloudFreeClipped_l7(i, roi)

In [ ]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


In [ ]:
# Greenest Pixel Composite
def greenestPixel_l8(year, area):
    # Get raw Landsat8 data input for simple composite
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
    
    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-07-01', str(year)+'-09-30'

    # Filter Landsat8 collection by date and by ROI
    collection = l8.filterDate(d1, d2).filterBounds(area)
    print('collectionl8', type(collection))
    
#     # Map over NDVI function
#     withNDVI = collection.map(addNDVI)
    
    
    # Create cloud-free composite with default params
    composite = ee.Algorithms.Landsat.simpleComposite(collection)
    print('compositel8', type(composite))
    
    ndvi = addNDVI(composite).select('NDVI')
    
    # Clip to ROI
    clippedComposite = ndvi.clip(area)
    print('clippedCompositel8', type(clippedComposite))
    
    ndviParams = {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    
    # Display the clipped composite as RGB
    Map.addLayer(clippedComposite, ndviParams, str(year))
    
    return clippedComposite



##### spectral unmixing

In [ ]:
l8_2016 = cloudFreeClipped_l8(2016, roi)

In [ ]:
# Let's calculate spectral unmixing for a single landsat8 ee.Image object
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'];
image = l8_2016.select(bands)
Map.setCenter(-104.9903, 39.7392, 10) # Denver, CO
Map.addLayer(image, {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 128}, 'Denver NIR image')

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])
Map.addLayer(fractions, {}, 'Denver unmixed')
    

In [ ]:
# Add 30m DEM to map
dataset = ee.Image('USGS/SRTMGL1_003')
elevation = dataset.select('elevation').clip(roi)
slope = ee.Terrain.slope(elevation)
Map.setCenter(-104.9903, 39.7392, 10)
Map.addLayer(slope, {min: 0, max: 60}, 'slope')

In [ ]:
Map